In [1]:
import torch
import numpy as np

In [3]:
def knn(data, k=5)->torch.Tensor:
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      k: int number of neighbours

    Returns:
      idx: shape:(batch_size, num_points, nums_neighours,)
    """
    dists_matrix = torch.cdist(data, data)
    #print(dists_matrix.shape)
    _, idx = dists_matrix.topk(k+1, dim=-1, largest=False)  # +1 the point itself is included
    return idx[...,1:] # not include the point itself



def get_edge_feature(point_cloud, idx=None, k=20,device="cpu"):
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      idx: (batch_size, num_points, neighbours)
      k: int
      device: cpu/cuda

    Returns:
      features: (batch_size, num_dims ,num_points, k)
    """
    point_cloud = point_cloud.to(device)
    batch_size = point_cloud.shape[0]
    num_points = point_cloud.shape[1]

    if(idx==None):
        idx = knn(point_cloud,k=k) # (batch_size, num_points, nums_neighours,)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points # create the base index for mapping
    idx = idx.to(device=device)
    idx = idx + idx_base #[0...0...0]->[0...100...200]
    idx=idx.view(-1) # flatten it -> tensor([  0,  56,  25,  ..., 225, 222, 271], device='cuda:0') e.g: [K01,K02,K03,K11,K12,K13...] shape = (B*N*K) 
   
    num_dims = point_cloud.shape[2]

    # feature : turn neighbour index in idx to coordinate
    feature = point_cloud.view(batch_size*num_points, -1)[idx, :] # feature : B*N*F -> BN * F -> (B*N*K) * F
    # feature : reshape into (Batch_size * Num_points *Nums_neigbours * Features)
    feature = feature.view(batch_size, num_points, k, num_dims)
    
    # pointcloud : create replicate of the self point up to k for matching feature - size B*N*K(repeated)*F 
    point_cloud = point_cloud.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1) 

    # feature size B*N*K*F -> B*N*K*2F (feature-x || x)
    feature = torch.cat((feature-point_cloud, point_cloud), dim=3)

    # (B * 2F * N * K) for later conv each coordinate(F)
    feature=feature.permute(0,3,1,2).contiguous()

    return feature

# Example usage:
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
#neighbors = knn(data, k=4)
edges= get_edge_feature(data)
print(edges.shape)
print(type(edges))

torch.Size([3, 6, 100, 20])
<class 'torch.Tensor'>


### **Edgeconv**

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EdgeConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_neighbours=20,device="cpu"):
        """Setup EdgeConv
        Args:
        in_channels: int
        out_channels: int
        num_neighbours: int
        """
        super(EdgeConv, self).__init__()
        self.device=device
        self.k= num_neighbours
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels*2, out_channels=out_channels, kernel_size=1, bias=False,device=self.device),
            nn.BatchNorm2d(out_channels,device=self.device),
            nn.LeakyReLU(negative_slope=0.2)
        )

    def forward(self,x):
        """Setup EdgeConv
        Args:
        x: shape - (batch_size, num_points, num_dims)

        Returns:
        features: (batch_size, num_dims, num_points, num_neigbours)
        """
        x = get_edge_feature(x, k=self.k,device=self.device) #(batch_size, num_points, dim) -> (batch_size, dim*2, num_points ,k)
        x = self.conv(x)
        # for each point pick the largest k (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)
        x = x.max(dim=-1, keepdim=False)[0]
        x = x.permute(0,2,1).contiguous()
        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
conv = EdgeConv(3, 64,device=device)
out = conv(data)
print("out.shape=", out.shape)

out.shape= torch.Size([3, 100, 64])


### **DGCNN (Classification)**

In [5]:
class DGCNN(nn.Module):
    def __init__(self, num_neighbours=20,out_channels=40,dropout_rate =0.3,device="cpu"):
        super(DGCNN,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)
        self.edgeConv3 = EdgeConv(in_channels=128,out_channels=256,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=512,out_channels=1024,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(2048, 512, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(512,device=device)
        self.drop1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(512, 256, bias=False,device=device)
        self.bn2 = nn.BatchNorm1d(256,device=device)
        self.drop2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)
        x3=self.edgeConv3(x2)

        x=torch.cat((x0,x1,x2,x3),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128+256) -> (batch_size, num_points, emb_dims(1024))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        maxPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        avgPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        x=torch.cat((maxPoolX,avgPoolX),1) #(batch_size, 2048)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x=self.drop1(x)
        x= F.leaky_relu(self.bn2(self.linear2(x)))
        x=self.drop2(x)
        x= self.linear3(x)
        # output

        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
print("data shape: ", data.shape)
print(device)
dgcnn = DGCNN(device=device)
out = dgcnn(data)

print("out.shape=", out.shape)

data shape:  torch.Size([3, 100, 3])
cuda
out.shape= torch.Size([3, 40])


## Dataset Part

In [9]:
import os
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset
import json
import warnings
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc

class ShapeNet(Dataset):
    def __init__(self,root, npoints=1024, split='train', class_choice=None, normal_channel=False):
        self.npoints = npoints
        self.root = root
        self.catfile = os.path.join(self.root, 'synsetoffset2category.txt')
        self.cat = {}
        self.normal_channel = normal_channel


        with open(self.catfile, 'r') as f:
            for line in f:
                ls = line.strip().split()
                self.cat[ls[0]] = ls[1]
        self.cat = {k: v for k, v in self.cat.items()}
        self.classes_original = dict(zip(self.cat, range(len(self.cat))))

        if not class_choice is  None:
            self.cat = {k:v for k,v in self.cat.items() if k in class_choice}
        # print(self.cat)

        self.meta = {}
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_train_file_list.json'), 'r') as f:
            train_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_val_file_list.json'), 'r') as f:
            val_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_test_file_list.json'), 'r') as f:
            test_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        for item in self.cat:
            # print('category', item)
            self.meta[item] = []
            dir_point = os.path.join(self.root, self.cat[item])
            fns = sorted(os.listdir(dir_point))
            # print(fns[0][0:-4])
            if split == 'trainval':
                fns = [fn for fn in fns if ((fn[0:-4] in train_ids) or (fn[0:-4] in val_ids))]
            elif split == 'train':
                fns = [fn for fn in fns if fn[0:-4] in train_ids]
            elif split == 'val':
                fns = [fn for fn in fns if fn[0:-4] in val_ids]
            elif split == 'test':
                fns = [fn for fn in fns if fn[0:-4] in test_ids]
            else:
                print('Unknown split: %s. Exiting..' % (split))
                exit(-1)

            # print(os.path.basename(fns))
            for fn in fns:
                token = (os.path.splitext(os.path.basename(fn))[0])
                self.meta[item].append(os.path.join(dir_point, token + '.txt'))

        self.datapath = []
        for item in self.cat:
            for fn in self.meta[item]:
                self.datapath.append((item, fn))

        self.classes = {}
        for i in self.cat.keys():
            self.classes[i] = self.classes_original[i]

        # Mapping from category ('Chair') to a list of int [10,11,12,13] as segmentation labels
        self.seg_classes = {'Earphone': [16, 17, 18], 'Motorbike': [30, 31, 32, 33, 34, 35], 'Rocket': [41, 42, 43],
                            'Car': [8, 9, 10, 11], 'Laptop': [28, 29], 'Cap': [6, 7], 'Skateboard': [44, 45, 46],
                            'Mug': [36, 37], 'Guitar': [19, 20, 21], 'Bag': [4, 5], 'Lamp': [24, 25, 26, 27],
                            'Table': [47, 48, 49], 'Airplane': [0, 1, 2, 3], 'Pistol': [38, 39, 40],
                            'Chair': [12, 13, 14, 15], 'Knife': [22, 23]}

        # for cat in sorted(self.seg_classes.keys()):
        #     print(cat, self.seg_classes[cat])

        self.cache = {}  # from index to (point_set, cls, seg) tuple
        self.cache_size = 20000


    def __getitem__(self, index):
        if index in self.cache:
            ppoint_set, cls, seg = self.cache[index]
        else:
            fn = self.datapath[index]
            cat = self.datapath[index][0]
            cls = self.classes[cat]
            cls = np.array([cls]).astype(np.int32)
            data = np.loadtxt(fn[1]).astype(np.float32)
            if not self.normal_channel:
                point_set = data[:, 0:3]
            else:
                point_set = data[:, 0:6]
            seg = data[:, -1].astype(np.int32)
            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, cls, seg)
        point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

        choice = np.random.choice(len(seg), self.npoints, replace=True)
        # resample
        point_set = point_set[choice, :]
        seg = seg[choice]

        return point_set, cls

    def __len__(self):
        return len(self.datapath)
_root='/kaggle/input/model4/shapenetcore_partanno_segmentation_benchmark_v0_normal/shapenetcore_partanno_segmentation_benchmark_v0_normal'
data = ShapeNet(npoints=1024,root=_root,split='test')
_data, label= data[0]
print(_data.shape)
print(label.shape)

(1024, 3)
(1,)


In [10]:
# import os
# import h5py
# import torch
# from torch.utils.data import Dataset

# train_list = ['ply_data_train0.h5', 'ply_data_train1.h5', 'ply_data_train2.h5', 'ply_data_train3.h5', 'ply_data_train4.h5', 'ply_data_train5.h5']
# test_list = ['ply_data_test0.h5', 'ply_data_test1.h5']
# val_list = ['ply_data_val0.h5']

# def make_data(mode='train', path='/kaggle/input/model4/shapenet_part_seg_hdf5_data/hdf5_data', num_point=1024):
#     datas = []
#     labels = []
#     if mode == 'train':
#         for file_list in train_list:
#             f = h5py.File(os.path.join(path, file_list), 'r')
#             datas.extend(f['data'][:, :num_point, :])
#             labels.extend(f['label'])
#             f.close()
#     elif mode == 'test':
#         for file_list in test_list:
#             f = h5py.File(os.path.join(path, file_list), 'r')
#             datas.extend(f['data'][:, :num_point, :])
#             labels.extend(f['label'])
#             f.close()
#     else:
#         for file_list in val_list:
#             f = h5py.File(os.path.join(path, file_list), 'r')
#             datas.extend(f['data'][:, :num_point, :])
#             labels.extend(f['label'])
#             f.close()
    
#     return datas, labels

# class PointDataset(torch.utils.data.Dataset):
#     def __init__(self, datas, labels):
#         super().__init__()
#         self.datas = datas
#         self.labels = labels
    
#     def __getitem__(self, index):
#         data = torch.tensor(self.datas[index].T.astype('float32'))
#         label = torch.tensor(self.labels[index].astype('int64'))
#         data = data.transpose(0, 1)
#         return data, label

#     def __len__(self):
#         return len(self.datas)

# datas, labels = make_data(mode='train', num_point=1024)
# train_dataset = PointDataset(datas, labels)
# _data, label= train_dataset[0]
# print(_data.shape)
# print(label.shape)
# datas, labels = make_data(mode='val', num_point=1024)
# val_dataset = PointDataset(datas, labels)
# _data, label= val_dataset[0]
# print(_data.shape)
# print(label.shape)
# datas, labels = make_data(mode='test', num_point=1024)
# test_dataset = PointDataset(datas, labels)
# _data, label= test_dataset[0]
# print(_data.shape)
# print(label.shape)

torch.Size([1024, 3])
torch.Size([1])
torch.Size([1024, 3])
torch.Size([1])
torch.Size([1024, 3])
torch.Size([1])


## Training part

In [10]:
#todo , not finish 
from torch.utils.data import DataLoader
import os
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import sklearn.metrics as metrics
def cal_loss(pred, gold, smoothing=True):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.cross_entropy(pred, gold, reduction='mean')

    return loss
def train(model, epochs, _lr, numPoints,batchSize,device):
    """train
        Args:
        model: classifier
        dataset: shape(batch_size,nums_point,dimemsion)
        optimizerSelect:1 for SGD, 0 for Adam
        epochs:train epochs
        device:
    """

    train_loader = DataLoader(ShapeNet(npoints=1024,root=_root,split='train'),
                              batch_size=batchSize, shuffle=True)

    test_loader = DataLoader(ShapeNet(npoints=1024,root=_root,split='test'), 
                             batch_size=batchSize, shuffle=True)


    #Try to load models
    model = nn.DataParallel(model)
    opt = optim.Adam(model.parameters(), lr=_lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(opt, epochs, eta_min=_lr)
    criterion = cal_loss
    best_test_acc = 0
    i=0
    for epoch in range(epochs):
        print("test1")
        scheduler.step()
        
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            preds = logits.max(dim=1)[1]
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
            print("test2")
        print("test3")
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,train_loss*1.0/count,metrics.accuracy_score
                                                    (train_true, train_pred),metrics.balanced_accuracy_score
                                                    (train_true, train_pred))
    
        print(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            batch_size = data.size()[0]
            logits = model(data)
            loss = criterion(logits, label)
            preds = logits.max(dim=1)[1]
            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())

        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,test_loss*1.0/count,test_acc,avg_per_class_acc)
        print(outstr)
        if test_acc >= best_test_acc:
            best_test_acc = test_acc
            BASE_DIR = os.getcwd()
            DATA_DIR = os.path.join(BASE_DIR, 'checkpoints')
            if not os.path.exists(DATA_DIR):
                os.mkdir(DATA_DIR)
                os.mkdir(os.path.join(DATA_DIR, 'models'))
            torch.save(model.state_dict(), '/kaggle/working/model_shape_5.t7')    
    
    
    return 0



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=DGCNN(device=device).to(device)
train(model,5,0.001,1024,8,device)

test1


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


RuntimeError: scatter(): Expected dtype int64 for index

## Run Code

In [ ]:
## Excute DGCNN
